In [3]:
import oracledb as db
from sqlalchemy import create_engine, Connection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

from datetime import timedelta, date
from dateutil.relativedelta import relativedelta
from time import sleep

from getpass import getpass
dsn = 'BIFC'
user='DANIEL_MELO'
password= getpass('senha usuario: ')

inicio_ano = date(2023, 1, 1) #date(2025, 12, 1) 
periods = (date.today().year - inicio_ano.year) * 12 + (date.today().month - inicio_ano.month) # relativedelta(date.today(), inicio_ano).months 
range_date = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS')

In [3]:
periods

1

In [4]:
range_date = range_date.strftime('%Y%m')
range_date

Index(['202512'], dtype='object')

In [5]:
range_date_start = pd.date_range(start = inicio_ano, periods= periods, freq = 'MS').strftime('%d-%m-%Y')
range_date_end = pd.date_range(start = inicio_ano, periods= periods, freq = 'M').strftime('%d-%m-%Y')

In [6]:
range_date_end

Index(['31-12-2025'], dtype='object')

## Base Demanda de Produtos:

In [ ]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
            f"""
             -- DEMANDA DE PRODUTO POR LOJA!
                WITH CALENDARIO AS (
                    SELECT 
                        TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + LEVEL - 1 AS data
                    FROM dual
                    CONNECT BY LEVEL <= TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY') - TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + 1
                order by 1 asc
                )

                -- CTE para garantir todas as combinações de data, produto e loja
                , BASE AS (
                    SELECT 
                        C.DATA,
                        Z.CODIGO,
                        Z.COD_EMPRESA
                        
                    FROM CALENDARIO C
                    CROSS JOIN (
                        SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
                        FROM PRODB B
                        JOIN PROD P ON P.CODIGO =B.CODIGO
                        
                        WHERE P.ATIVO = 'S'
                        AND NVL(P.LG_SERVICO, 'N') = 'N'
                        AND P.LINHA = 'UAA'
                        AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')
                    ) Z
                )


                ,PADRAO_COMPRA AS (
                    SELECT DISTINCT M.CODIGO, M.COD_EMPRESA, NVL(M.PADRAO_COMPRA_COD_EMPRESA, M.COD_EMPRESA) AS PADRAO_COMPRA
                    FROM PROD_MIX M
                    WHERE M.ID_LOJA = 0
                )


                SELECT B.DATA, B.COD_EMPRESA, B.CODIGO,  
                    V.UND_VENDA, NVL(V.QTD_VENDIDA,0) as QTD_VENDA, 
                    M.PADRAO_COMPRA
                    
                FROM BASE B
                LEFT JOIN PADRAO_COMPRA M ON B.COD_EMPRESA = M.COD_EMPRESA AND B.CODIGO = M.CODIGO
                LEFT JOIN (
                SELECT NVL(R.DATA_PEDIDO, S.DATAEMISSAO) AS DATA,
                    P.CODIGO,
                    S.COD_EMPRESA,
                    S.UNVENDA3 AS UND_VENDA,
                    SUM(S.QTSV * NVL(S.CONVERSAO,1)) AS QTD_VENDIDA
                    
                    
                FROM SAIDM_IT S
                JOIN SAIDM SM
                ON SM.NUMNOTA = S.NUMNOTA
                AND S.COD_EMPRESA = SM.COD_EMPRESA
                JOIN PROD P
                ON SUBSTR(S.PRODUTO,1,LENGTH(S.PRODUTO)-2) = P.CODIGO



                LEFT JOIN SAID_IT_RENTA R
                ON R.COD_EMPRESA = S.COD_EMPRESA
                AND R.NUMNOTA = S.NUMNOTA
                AND R.PRODUTO = S.PRODUTO
                AND R.DATAEMISSAO = S.DATAEMISSAO
                AND R.TIPO_CUSTO = 'P'

                WHERE 1=1
                AND P.LINHA = 'UAA'
                AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')
                AND P.ATIVO ='S'
                AND S.TIPOSAIDA = '1'
                AND ((NVL(SM.STATUS, '0') != 'C' AND NVL(SM.STATUS, '0')  != 'D') OR
                    (NVL(SM.STATUS, '0')  = 'D' AND NVL(SM.CODENTREG, 0) != 101))
                    
                AND S.DATAEMISSAO BETWEEN TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') AND TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY')
                
                GROUP BY NVL(R.DATA_PEDIDO, S.DATAEMISSAO),
                    P.CODIGO,
                    S.COD_EMPRESA,
                    S.UNVENDA3
                    
                ORDER BY COD_EMPRESA, CODIGO, DATA 
                ) V ON B.DATA = V.DATA AND B.COD_EMPRESA = V.COD_EMPRESA AND B.CODIGO = V.CODIGO
            """
            
            df_ = pd.read_sql(script[i], con = conn)
        
            # Verifica se é a primeira iteração. Caso for, inicializa um DF vazio
            if i == 0:
                df = df_
            else:
                # Empilha os resultados de cada mês
                df = pd.concat([df, df_], ignore_index = True, axis = 0)
    
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle
    conn.close()
    print('Conxeão Fechada')

100%|██████████| 1/1 [05:28<00:00, 328.56s/it]

Dados Coletados
Conxeão Fechada


## Base Estoque dia-dia

In [ ]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
            f"""
             -- ESTOQUE DIARIO
            WITH CALENDARIO AS (
                SELECT 
                    TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + LEVEL - 1 AS data
                FROM dual
                CONNECT BY LEVEL <= TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY') - TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') + 1
            order by 1 asc
            )

            -- CTE para garantir todas as combinações de data, produto e loja
            , BASE AS (
                SELECT 
                    C.DATA,
                    Z.CODIGO,
                    Z.COD_EMPRESA
                                    
                FROM CALENDARIO C
                CROSS JOIN (
                    SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
                    FROM PRODB B
                    JOIN PROD P ON P.CODIGO =B.CODIGO
                                    
                    WHERE P.ATIVO = 'S'
                    AND NVL(P.LG_SERVICO, 'N') = 'N'
                    AND P.LINHA = 'UAA'
                    AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')
                ) Z
            )

            SELECT B.DATA, 
                B.COD_EMPRESA, 
                B.CODIGO,  
                E.ESTOQUE_DIARIO

                
            FROM BASE B
            LEFT JOIN (
                        SELECT B.DATA,
                            B.COD_EMPRESA,
                            B.CODIGO,
                            (NVL(qtde1, 0)+   NVL(qtde2, 0)+   NVL(qtde3, 0)+  NVL(qtde4, 0)+  NVL(qtde5, 0) +
                            NVL(qtde6, 0)+   NVL(qtde7, 0)+   NVL(qtde8, 0)+  NVL(qtde9, 0)+  NVL(qtde10, 0)+
                            NVL(qtde11, 0)+  NVL(qtde12, 0)+  NVL(qtde13, 0)+ NVL(qtde14, 0)+ NVL(qtde15, 0)+
                            NVL(qtde16, 0)+  NVL(qtde17, 0)+  NVL(qtde18, 0)+ NVL(qtde19, 0) + NVL(QTDE20,0)) ESTOQUE_DIARIO
                        FROM PRODB_DIARIO_{range_date[i]} B
                        JOIN PROD P ON P.CODIGO = B.CODIGO 
                        WHERE P.ATIVO ='S'
                        AND P.LINHA = 'UAA'
                        AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')

                        
                        UNION ALL
                                
                        SELECT F.DT_REFERENCIA,
                            DF.CD_EMPRESA ,
                            DP.CD_PRODUTO AS CODIGO, 
                            F.ESTOQUE AS ESTOQUE_DIARIO
                        
                        FROM FATO_COMESTQ_{range_date[i]} F
                        JOIN DIM_PRODUTO DP ON F.DIM_PRODUTO = DP.DIM_PRODUTO
                        JOIN DIM_FILIAL DF ON DF.DIM_FILIAL = F.DIM_FILIAL
                        JOIN PROD P ON P.CODIGO = DP.CD_PRODUTO
                        
                        WHERE P.ATIVO ='S'
                        AND P.LINHA = 'UAA'
                        AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')


            ) E ON B.DATA = E.DATA AND B.COD_EMPRESA = E.COD_EMPRESA AND B.CODIGO = E.CODIGO
            

            """
            
            df_estoque_mes = pd.read_sql(script[i], con = conn)
        
            # Verifica se é a primeira iteração. Caso for, inicializa um DF vazio
            if i == 0:
                df_est = df_estoque_mes
            else:
                # Empilha os resultados de cada mês
                df_est = pd.concat([df_est, df_estoque_mes], ignore_index = True, axis = 0)
    
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle
    conn.close()
    print('Conxeão Fechada')

100%|██████████| 1/1 [01:29<00:00, 89.41s/it]

Dados Coletados
Conxeão Fechada


In [11]:
base_demanda = df.merge(df_est, how='left', on = ['data', 'cod_empresa', 'codigo'], validate = '1:1')
base_demanda.head()

,data,cod_empresa,codigo,und_venda,qtd_venda,padrao_compra,estoque_diario
0,2025-12-20,1,117790,UN,1,80.0,0.0
1,2025-12-03,1,158374,UN,1,80.0,17.0
2,2025-12-17,1,158374,UN,1,80.0,16.0
3,2025-12-26,1,158374,UN,2,80.0,14.0
4,2025-12-29,1,158374,UN,2,80.0,12.0


In [ ]:
for i in tqdm(range(0,periods,1)):
    base_demanda.\
    loc[(base_demanda['data'] >= range_date_start[i]) & (base_demanda['data'] <= range_date_end[i])].\
    to_csv(f'./Data/Demanda/demanda_{range_date_start[i]}',  index=False, encoding = 'utf-8')

100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


## Pedidos Fornecedor

In [ ]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}
        for i in tqdm(range(0,periods,1)):
            if i>0:
                sleep(60)
    
            script[i] = \
              f"""
                   -- ESTOQUE_COMPRAS
                    WITH ESCOPO AS (
                    SELECT DISTINCT P.CODIGO, B.COD_EMPRESA
                            FROM PRODB B
                            JOIN PROD P ON P.CODIGO =B.CODIGO
                                                    
                            WHERE P.ATIVO = 'S'
                            AND NVL(P.LG_SERVICO, 'N') = 'N'
                            AND P.LINHA = 'UAA'
                            AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI')
                    )
                                
                    -- CNPJ  SKU Filial de Entrega Tipo_Destino  Lead_Time_Dias  Ciclo_Pedido_Dias Faturamento Mínimo  Multiplo_Palete Multiplo_Carreta  Custo_Unitario  Custo_Frete_Palete  Estoque_Disponivel  Estoque_Transito

                    SELECT 
                        NVL(E2.DATAENTRADA, E.DATAENTRADA) AS DATA_ENTRADA,
                        E.NUMPEDIDO,
                        P.FORNECEDOR AS CNPJ,
                        EI.PRODUTO,
                        DECODE(NVL(E.CGC_TRANSF_AUTO, ' '), ' ', E.COD_EMPRESA, PA.COD_EMPRESA) AS FILIAL_ENTREGA, 
                        CASE WHEN DECODE(NVL(E.CGC_TRANSF_AUTO, ' '), ' ', E.COD_EMPRESA, PA.COD_EMPRESA) >= 80
                            THEN 'CD'
                            ELSE 'LOJA'
                            END AS TIPO_DESTINO,
                        (NVL(E2.DATAENTRADA, E.DATAENTRADA) - E.DATAPEDIDO) AS LEAD_TIME_DIAS,
                        NULL AS CICLO_PEDIDO,
                        F.FAT_MINIMO AS FATURAMENTO_MINIMO,    
                        EI.PRECO_UNIT AS PRECO_UNIT,
                        EI.QUANTIDADE AS QTD_ENTREGUE,
                        EI.CONVERSAO AS CONVERSAO_UN_ESTOQUE,
                        NULL AS ESTOQUE_DISPONIVEL,
                        EI.QUANTIDADE * NVL(DECODE(EI.CONVERSAO,0,1,EI.CONVERSAO), 1) AS QTD_CONVERTIDA
                                    
                    FROM ENTR_IT EI

                    JOIN ESCOPO ES ON ES.CODIGO = EI.PRODUTO AND ES.COD_EMPRESA = EI.COD_EMPRESA

                    JOIN ENTR E 
                    ON E.CGC_EMITENTE = EI.CGC_EMITENTE 
                    AND E.NUMNOTA      = EI.NUM_NOTA
                    AND E.COD_EMPRESA  = EI.COD_EMPRESA
                    AND E.SERIE        = EI.SERIE 

                    LEFT JOIN (
                        SELECT DISTINCT P26.COD_EMPRESA, P26.CGC
                        FROM PARAM026 P26
                        
                    ) PA ON PA.CGC = E.CGC_TRANSF_AUTO

                    JOIN PROD P ON P.CODIGO = EI.PRODUTO
                    JOIN FORN F ON F.CGC = P.FORNECEDOR
                    
                    LEFT JOIN (
                        SELECT DISTINCT V.CHAVE_NFE_ORIGEM_TRANSF, V.CHAVE_NFE, V.COD_EMPRESA
                        FROM VCEGA V
                        WHERE 1=1
                        AND NVL(V.STATUS,' ') = 'B'
                        AND V.TIPO = '11'
                        AND NVL(V.CHAVE_NFE_ORIGEM_TRANSF, ' ') <> ' '
                    
                    ) V ON E.CHAVE_NFE = V.CHAVE_NFE_ORIGEM_TRANSF    -- AQUI EU PEGO SE A ENTRADA SE REFERE A ORIGEM DE UMA TRANSFERÊNCIA AUTOMÁTICA. 

                    LEFT JOIN ENTR E2 ON V.CHAVE_NFE = E2.CHAVE_NFE AND E2.TIPO = '11'   -- USO ESSE JOIN PRA SABER A DATA ENTRADA QUE A MERCADORIA DEU NO DESTINO FINAL.
                    
                    WHERE 1=1
                    AND NVL(E2.DATAENTRADA, E.DATAENTRADA) BETWEEN TO_DATE('{range_date_start[i]}', 'DD-MM-YYYY') AND TO_DATE('{range_date_end[i]}', 'DD-MM-YYYY')
                    AND E.TIPO = '1'
                    AND NVL(E.STATUS, ' ') <> 'C'
                  """
            
            ped_fornecedor = pd.read_sql(script[i], con = conn)

            ped_fornecedor.to_csv(f'./Data/pedidos_fornecedor/pedidos_fornecedor_{range_date_start[i]}.csv',  index=False, encoding = 'utf-8')
            
    
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle
    conn.close()
    print('Conxeão Fechada')

## Transferencias CD

In [ ]:
try:
    engine = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")
    with engine.connect() as conn:
        # Objeto que vai armazenar consultas:
        script = {}

        script = \
            f"""
            SELECT S.MOTIVO, 
                    S.NU_PEDIDO,
                    S.DT_MOVIMENTO,
                    S.COD_EMPRESA_ORIGEM,
                    S.COD_EMPRESA,
                    P.CODIGO,
                    S.DT_MOVIMENTO  - S.DT_SOLI AS LEAD_TIME_DIAS,
                    S.QTD_ATENDIDA,
                    B.CUE AS CUSTO_UNIT
                                            
                FROM SOLI S
                JOIN PROD P ON P.CODIGO = SUBSTR(S.COD_PRODUTO,1, LENGTH(S.COD_PRODUTO)-2)
                JOIN PRODB B ON B.CODIGO = P.CODIGO AND B.COD_EMPRESA = S.COD_EMPRESA

                WHERE NVL(S.MOTIVO, ' ') NOT LIKE ('TRANSF%AUTO%')
                AND NVL(S.STATUS, ' ') NOT IN ('E')
                AND S.COD_EMPRESA_ORIGEM >= 80
                AND P.ATIVO = 'S'
                AND NVL(P.LG_SERVICO, 'N') = 'N'
                AND P.LINHA = 'UAA'
                AND P.FANTAS IN ('LORENZETTI SP', 'ZAGONEL', 'FAME', 'HYDRA', 'FIRENZI') 
                AND NVL(S.MOTIVO_CANC, 0) = 0
                AND S.QUANTIDADE >0  

                ORDER BY 1 ASC
  
            """
        
        transferencias = pd.read_sql(script, con = conn)    
    
except Exception as e:
    print(f'Erro: {e}')
    
else:
    print("Dados Coletados")
    
finally:
    # Desconectando conexão com o Oracle
    
    conn.close()
    print('Conxeão Fechada')

In [ ]:
transferencias = transferencias.merge(df_est, how='left', on = ['data', 'cod_empresa', 'codigo'], validate = '1:1')

## Campanhas

In [ ]:
from sqlalchemy import create_engine,text, types
import pandas as pd
from pandas import read_sql
from datetime import datetime,timedelta
from getpass import getpass
from dateutil.relativedelta import relativedelta
import gc

In [ ]:
def get_sqlalchemy_dtypes(df):
    type_mapping = {
        "object": types.String,
        "int64": types.Integer,
        "float64": types.Float,
        "datetime64[ns]": types.DateTime,
        "datetime64[ns, UTC]": types.DateTime,
        "bool": types.Boolean
    }
    dtypes = {}
    for col, dtype in df.dtypes.items():
        sql_type = type_mapping.get(str(dtype), types.String)
        dtypes[col] = sql_type
    return dtypes

engine_bifc = create_engine(f"oracle+oracledb://{user}:{password}@{dsn}")


In [ ]:
#Campanhas

query_linha = f""" 
SELECT DISTINCT 
       F.COD_EMPRESA,
       F.COD_CAMPANHA_FILIAL AS CODIGO_CAMPANHA,
       F.NM_CAMPANHA_FILIAL AS NOME_CAMPANHA,
       F.DT_LOJA_INICIO AS start_date,
       F.DT_LOJA_FINAL AS end_date,
       M.NM_CAMPANHA AS type,
       T.NM_TIPO_CAMPANHA AS subtype


FROM SFC_CAMP_FILIAL F
LEFT JOIN SFC_CAMP_MASTER M ON F.COD_CAMPANHA_MASTER = M.COD_CAMPANHA_MASTER
LEFT JOIN SFC_CAMP_TIPO T ON M.COD_TIPO_CAMPANHA = T.COD_TIPO_CAMPANHA
WHERE F.DT_LOJA_INICIO > ADD_MONTHS(SYSDATE, -36)
  AND F.DT_LOJA_FINAL <= SYSDATE

ORDER BY START_DATE ASC

    """

try:
    df_mes = pd.read_sql(text(query_linha), engine_bifc)
    df_mes.to_csv(r"./Data/campanhas/campanhas.csv", index=False, encoding = 'utf-8')
    del df_mes        
except Exception as e:
    print(f"❌ Erro ao acessar {query_linha}: {e}")


    
engine_bifc.dispose()

In [ ]:
#Produtos_locais_campanhas

query_linha = f""" 

SELECT CAST(CP.COD_CAMPANHA_FILIAL AS VARCHAR(20)) AS CODIGO_CAMPANHA,
       P.CODIGO AS CODIGO_PROD,
       CAST(CP.COD_EMPRESA AS VARCHAR(4)) AS COD_EMPRESA,
       ROUND(CP.PRVD1_CAMPANHA * ( 1 -CP.DESCONTO/100), 2) AS PRECO_CAMPANHA

FROM SFC_CAMP_PROD CP
JOIN PROD P ON P.CODIGO = CP.COD_PRODUTO

WHERE EXISTS ( 
      SELECT 1
        FROM SFC_CAMP_FILIAL F
        LEFT JOIN SFC_CAMP_MASTER M ON F.COD_CAMPANHA_MASTER = M.COD_CAMPANHA_MASTER
        LEFT JOIN SFC_CAMP_TIPO T ON M.COD_TIPO_CAMPANHA = T.COD_TIPO_CAMPANHA
        WHERE F.DT_LOJA_INICIO > ADD_MONTHS(SYSDATE, -36)
          AND F.DT_LOJA_FINAL <= SYSDATE
          AND F.COD_EMPRESA = CP.COD_EMPRESA
          AND F.COD_CAMPANHA_FILIAL = CP.COD_CAMPANHA_FILIAL
)
AND CP.PRVD1_CAMPANHA > 0

    """

try:
    df_mes = pd.read_sql(text(query_linha), engine_bifc)
    df_mes.to_csv(r"./Data/campanhas/produtos_campanha.csv", index=False, encoding = 'utf-8')
    del df_mes             
except Exception as e:
    print(f"❌ Erro ao acessar {query_linha}: {e}")


    
engine_bifc.dispose()

In [18]:
pd.read_csv('C:\\Users\\daniel.melo\\Desktop\\Planejamento & Previsão de Demanda\\Data\\demanda_01-12-2025').to_excel('amostra.xlsx')